In [1]:
#Importing the library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [37]:
#Importing the data

data = pd.read_csv('dataset-grid-toplology-1.csv',sep="\t")
data.head()

,No,Time,From,To,sourceaddress,destinationaddress,edgenodeaddress,INSTANCE_ID,rank,rank_min,...,hops,distance,node_type,node_honest_level,energy_efficiency_OF,lattency_min_OF,pdr_OF,congestion_OF,throughput_opt_OF,Data
0,2,00:03.044,1,[25 d],fe80::212:74011101,fe80::212:74033303,fe80::212:74011101,87bb925b-d3ff-46a6-b339-7d48b8993bec,1024.0,256,...,2,-12,EDGE_NODE,793,-0.000479,-1737107271769,0.01,0.25,0.044803,97: 0x41D88ACD ABFFFF01 01010001 7412007A 3B3A...
1,3,00:03.993,20,[20 d],fe80::212:7420202020,fe80::212:74044404,fe80::212:74000101,bd86a2f6-b3d6-4c67-a9d3-2a42904c243c,NaN,256,...,2,128,CHILD_NODE,788,-0.000417,-1737107271609,0.01,0.25,0.043178,102: 0x61DC18CD AB020202 00027412 00141414 001...
2,4,00:03.993,24,0,fe80::212:7424242424,0,0,0233072f-b8cd-4189-818e-2a4e2b8f1ae7,65535.0,256,...,1,256,CHILD_NODE,768,-0.000416,0,NaN,0.00,0.065104,102: 0x61DC18CD AB020202 00027412 00181818 001...
3,5,00:03.997,42,[10 d],fe80::212:7442424242,fe80::212:74044404,fe80::212:74000101,c0a88d50-730f-4c1a-b21e-83c23f59d9b0,NaN,256,...,1,128,CHILD_NODE,778,-0.000417,-1737107271609,0.01,0.25,0.041876,102: 0x61DC18CD AB020202 00027412 002A2A2A 002...
4,6,00:03.997,19,0,fe80::212:7419191919,0,0,3c256cd4-9b1b-43e3-ab9d-a61f12a776ce,65535.0,256,...,1,256,CHILD_NODE,768,-0.000417,0,NaN,0.00,0.065104,102: 0x61DC18CD AB020202 00027412 00131313 001...


In [3]:
# Profile of Data - No. of Rows,Columns,Data Type,Data Byte,Null Values

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33889 entries, 0 to 33888
Data columns (total 34 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   No                    33889 non-null  object 
 1   Time                  33889 non-null  object 
 2   From                  33889 non-null  int64  
 3   To                    33889 non-null  object 
 4   sourceaddress         33889 non-null  object 
 5   destinationaddress    33889 non-null  object 
 6   edgenodeaddress       33889 non-null  object 
 7   INSTANCE_ID           33889 non-null  object 
 8   rank                  32541 non-null  float64
 9   rank_min              33889 non-null  int64  
 10  rank_max              33889 non-null  int64  
 11  node_id               33889 non-null  int64  
 12  ver                   33889 non-null  int64  
 13  ver_min               33889 non-null  int64  
 14  ver_max               33889 non-null  int64  
 15  ver_diff           

In [39]:
#Changing the column name to lower

data.columns = data.columns.str.lower()

In [5]:
data.columns

Index(['no', 'time', 'from', 'to', 'sourceaddress', 'destinationaddress',
       'edgenodeaddress', 'instance_id', 'rank', 'rank_min', 'rank_max',
       'node_id', 'ver', 'ver_min', 'ver_max', 'ver_diff', 'sending_time',
       'sending_rate', 'delta_time', 'received_packets', 'forward_packets',
       'drop_count', 'energy_consumption', 'etx', 'hops', 'distance',
       'node_type', 'node_honest_level', 'energy_efficiency_of',
       'lattency_min_of', 'pdr_of', 'congestion_of', 'throughput_opt_of',
       'data'],
      dtype='object')

In [6]:
#Null Features

na_features = [feature for feature in data.columns if data[feature].isnull().sum()>1]
for features in na_features:
    print(f"{features}:{data[features].isnull().mean()*100: .3f}% missing value")


rank: 3.978% missing value
sending_time: 100.000% missing value
pdr_of: 20.780% missing value


In [41]:
data.drop(['sending_time'],axis = 1,inplace = True)
data['rank'] = data['rank'].fillna(data['rank'].mode()[0])
data['pdr_of'] = data['pdr_of'].fillna(data['pdr_of'].mode()[0])

In [45]:
data.isnull().sum().sum()

0

In [7]:
# Duplicated value

data[data.duplicated()]

,no,time,from,to,sourceaddress,destinationaddress,edgenodeaddress,instance_id,rank,rank_min,...,hops,distance,node_type,node_honest_level,energy_efficiency_of,lattency_min_of,pdr_of,congestion_of,throughput_opt_of,data


In [53]:
#Numerical Features

numerical_feature = [feature for feature in data.columns if data[feature].dtype != 'O']
print("No. of Numerical Column: ",len(numerical_feature))
data[numerical_feature].head()

No. of Numerical Column:  24


,from,rank,rank_min,rank_max,node_id,ver,ver_min,ver_max,ver_diff,sending_rate,...,energy_consumption,etx,hops,distance,node_honest_level,energy_efficiency_of,lattency_min_of,pdr_of,congestion_of,throughput_opt_of
0,1,1024.0,256,2048,1,240,0,240,240,2,...,3088,372,2,-12,793,-0.000479,-1737107271769,0.01,0.25,0.044803
1,20,1280.0,256,2048,20,240,0,240,240,2,...,3400,386,2,128,788,-0.000417,-1737107271609,0.01,0.25,0.043178
2,24,65535.0,256,2048,24,240,0,240,240,2,...,3401,256,1,256,768,-0.000416,0,0.01,0.00,0.065104
3,42,1280.0,256,2048,42,240,0,240,240,2,...,3400,398,1,128,778,-0.000417,-1737107271609,0.01,0.25,0.041876
4,19,65535.0,256,2048,19,240,0,240,240,2,...,3400,256,1,256,768,-0.000417,0,0.01,0.00,0.065104


In [51]:
# Categorical Columns

categorical_feature = [feature for feature in data.columns if data[feature].dtype == 'O']
print("No. of Categorical Column: ",len(categorical_feature))
data[categorical_feature].head()

No. of Categorical Column:  9


,no,time,to,sourceaddress,destinationaddress,edgenodeaddress,instance_id,node_type,data
0,2,00:03.044,[25 d],fe80::212:74011101,fe80::212:74033303,fe80::212:74011101,87bb925b-d3ff-46a6-b339-7d48b8993bec,EDGE_NODE,97: 0x41D88ACD ABFFFF01 01010001 7412007A 3B3A...
1,3,00:03.993,[20 d],fe80::212:7420202020,fe80::212:74044404,fe80::212:74000101,bd86a2f6-b3d6-4c67-a9d3-2a42904c243c,CHILD_NODE,102: 0x61DC18CD AB020202 00027412 00141414 001...
2,4,00:03.993,0,fe80::212:7424242424,0,0,0233072f-b8cd-4189-818e-2a4e2b8f1ae7,CHILD_NODE,102: 0x61DC18CD AB020202 00027412 00181818 001...
3,5,00:03.997,[10 d],fe80::212:7442424242,fe80::212:74044404,fe80::212:74000101,c0a88d50-730f-4c1a-b21e-83c23f59d9b0,CHILD_NODE,102: 0x61DC18CD AB020202 00027412 002A2A2A 002...
4,6,00:03.997,0,fe80::212:7419191919,0,0,3c256cd4-9b1b-43e3-ab9d-a61f12a776ce,CHILD_NODE,102: 0x61DC18CD AB020202 00027412 00131313 001...


In [55]:
#Statistical Analysis

print("Numerical Statistical Analysis")
print(data[numerical_feature].describe().T)
print("=" * 90)
print("Categorical Statistical Analysis")
print(data[categorical_feature].describe().T)

Numerical Statistical Analysis
                        count          mean           std           min  \
from                  33889.0  2.506689e+01  1.533382e+01  1.000000e+00   
rank                  33889.0  1.316538e+04  2.510750e+04  5.120000e+02   
rank_min              33889.0  2.560000e+02  0.000000e+00  2.560000e+02   
rank_max              33889.0  2.048000e+03  0.000000e+00  2.048000e+03   
node_id               33889.0  2.506689e+01  1.533382e+01  1.000000e+00   
ver                   33889.0  2.400000e+02  0.000000e+00  2.400000e+02   
ver_min               33889.0  0.000000e+00  0.000000e+00  0.000000e+00   
ver_max               33889.0  2.400000e+02  0.000000e+00  2.400000e+02   
ver_diff              33889.0  2.400000e+02  0.000000e+00  2.400000e+02   
sending_rate          33889.0  5.165186e+00  5.265408e-01  0.000000e+00   
delta_time            33889.0  2.166769e+07  7.549893e+04  2.156792e+07   
received_packets      33889.0  5.126548e+02  8.762151e+02  0.000000e+

In [57]:
# The unique values in the data

print("Unique values per column")
data.nunique().sort_values(ascending = True)

Unique values per column


ver                         1
ver_min                     1
ver_diff                    1
ver_max                     1
rank_max                    1
rank_min                    1
pdr_of                      1
congestion_of               2
distance                    3
hops                        3
edgenodeaddress             3
node_type                   3
sending_rate                6
rank                        6
destinationaddress         39
drop_count                 44
node_honest_level          44
node_id                    52
from                       52
sourceaddress              52
to                         87
etx                        89
throughput_opt_of          89
energy_efficiency_of      161
energy_consumption        161
delta_time                445
forward_packets          2065
lattency_min_of          3245
received_packets         3292
data                     7006
time                    31601
no                      33889
instance_id             33889
dtype: int

In [59]:
for i in data.columns:
    print(f"Unique value in {i}:")
    print(data[i].unique(),'\n')

Unique value in no:
['2' '3' '4' ... '34710' '34711' '34712'] 

Unique value in time:
['00:03.044' '00:03.993' '00:03.997' ... '03:35.881' '03:35.890'
 '03:35.894'] 

Unique value in from:
[ 1 20 24 42 19 21 28 32  4 17  9 41  6 12  3  2 34 33 46 29 51 15 37 14
 48 31 18 43 10 26 30 36 44 25  5 16  8 35 22 45 52 50 27 11 39 13 23 47
 40 38 49  7] 

Unique value in to:
['[25 d]' '[20 d]' '0' '[10 d]' '[13 d]' '[3 d]' '[35 d]' '[19 d]' '[4 d]'
 '[43 d]' '[39 d]' '44' '[22 d]' '[17 d]' '[31 d]' '[23 d]' '[28 d]'
 '[27 d]' '[38 d]' '[24 d]' '[44 d]' '[8 d]' '[30 d]' '[26 d]' '[12 d]'
 '[21 d]' '[5 d]' '[11 d]' '[6 d]' '[33 d]' '[7 d]' '[34 d]' '[41 d]'
 '[36 d]' '[42 d]' '[14 d]' '24,32' '26,34' '[37 d]' '41' '38,40' '[29 d]'
 '[32 d]' '[18 d]' '1,31' '[9 d]' '2,30' '31' '1,34' '24,30' '[16 d]'
 '[15 d]' '31,51' '32' '22' '2' '26' '34' '13,26' '1' '7,22' '49' '24'
 '36' '13,34' '9,34' '40' '5,36' '17' '2,44' '40,48' '5' '30' '48' '20,41'
 '37' '31,34' '21' '14' '7' '9' '46' '1,51' '34,47' 

In [61]:
# Discrete Feature

discrete_feature = [feature for feature in data.columns if data[feature].nunique()<100]
print("Discrete Variables Count:{}".format(len(discrete_feature)))
data[discrete_feature].head()

Discrete Variables Count:23


,from,to,sourceaddress,destinationaddress,edgenodeaddress,rank,rank_min,rank_max,node_id,ver,...,sending_rate,drop_count,etx,hops,distance,node_type,node_honest_level,pdr_of,congestion_of,throughput_opt_of
0,1,[25 d],fe80::212:74011101,fe80::212:74033303,fe80::212:74011101,1024.0,256,2048,1,240,...,2,25,372,2,-12,EDGE_NODE,793,0.01,0.25,0.044803
1,20,[20 d],fe80::212:7420202020,fe80::212:74044404,fe80::212:74000101,1280.0,256,2048,20,240,...,2,20,386,2,128,CHILD_NODE,788,0.01,0.25,0.043178
2,24,0,fe80::212:7424242424,0,0,65535.0,256,2048,24,240,...,2,0,256,1,256,CHILD_NODE,768,0.01,0.00,0.065104
3,42,[10 d],fe80::212:7442424242,fe80::212:74044404,fe80::212:74000101,1280.0,256,2048,42,240,...,2,10,398,1,128,CHILD_NODE,778,0.01,0.25,0.041876
4,19,0,fe80::212:7419191919,0,0,65535.0,256,2048,19,240,...,2,0,256,1,256,CHILD_NODE,768,0.01,0.00,0.065104


In [63]:
# Continous Feature

continous_feature = [feature for feature in data.columns if data[feature].nunique()>100]
print("Continous Variables Count:{}".format(len(continous_feature)))
data[continous_feature].head()

Continous Variables Count:10


,no,time,instance_id,delta_time,received_packets,forward_packets,energy_consumption,energy_efficiency_of,lattency_min_of,data
0,2,00:03.044,87bb925b-d3ff-46a6-b339-7d48b8993bec,21567917,3395,3420,3088,-0.000479,-1737107271769,97: 0x41D88ACD ABFFFF01 01010001 7412007A 3B3A...
1,3,00:03.993,bd86a2f6-b3d6-4c67-a9d3-2a42904c243c,21567917,3440,3460,3400,-0.000417,-1737107271609,102: 0x61DC18CD AB020202 00027412 00141414 001...
2,4,00:03.993,0233072f-b8cd-4189-818e-2a4e2b8f1ae7,21567917,0,0,3401,-0.000416,0,102: 0x61DC18CD AB020202 00027412 00181818 001...
3,5,00:03.997,c0a88d50-730f-4c1a-b21e-83c23f59d9b0,21567917,10,20,3400,-0.000417,-1737107271609,102: 0x61DC18CD AB020202 00027412 002A2A2A 002...
4,6,00:03.997,3c256cd4-9b1b-43e3-ab9d-a61f12a776ce,21567917,0,0,3400,-0.000417,0,102: 0x61DC18CD AB020202 00027412 00131313 001...


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=312e9fb3-d590-436d-aa3f-66956f58bdb5' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>